<a href="https://colab.research.google.com/github/MeidanGR/GooglePlayStore_Preprocessing_Python/blob/main/GooglePlayStore_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Play Store Data Preprocessing**
This data preprocessing progress has been established for an academic project of a BI course. The data will be analyzed within the project.

Major thanks to **Gautham Prakash** for publishing & maintenance this dataset, which you can find at https://www.kaggle.com/gauthamp10/google-playstore-apps


# **Google auth & libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [153]:
import pandas as pd
import numpy as np

# **CSV LOAD**


In [154]:
#Loading csv
df = pd.read_csv('/content/drive/My Drive/Google-Playstore.csv')

print("Dataframe original shape is", np.shape(df), end='\n\n')
print(df.info())
df.sample(2)

Dataframe original shape is (603047, 23)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603047 entries, 0 to 603046
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   App Name           603046 non-null  object 
 1   App Id             603047 non-null  object 
 2   Category           603047 non-null  object 
 3   Rating             599287 non-null  float64
 4   Rating Count       599287 non-null  float64
 5   Installs           602875 non-null  object 
 6   Minimum Installs   602875 non-null  float64
 7   Maximum Installs   603047 non-null  int64  
 8   Free               603047 non-null  bool   
 9   Price              603047 non-null  float64
 10  Currency           602866 non-null  object 
 11  Size               603047 non-null  object 
 12  Minimum Android    602556 non-null  object 
 13  Developer Id       603045 non-null  object 
 14  Developer Website  423783 non-null  object 
 15  Developer

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Currency,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
583986,Wave.io,studio.onepixel.wave,Arcade,4.9,12.0,"1,000+",1000.0,1691,True,0.0,USD,9.9M,4.0.3 and up,OnePixel Studio,https://onepixel.studio/,support@onepixel.studio,"Nov 27, 2018",14/12/2019 19:50,Everyone,http://onepixel.studio/privacypolicy/wave_io.html,True,False,False
363003,MPDX,org.mpdx,Communication,3.5,63.0,"5,000+",5000.0,5045,True,0.0,USD,15M,5.0 and up,Cru,http://www.mpdx.org,support@mpdx.org,"Sep 22, 2012",29/04/2020 23:28,Everyone,https://www.cru.org/about/privacy.html,False,False,False


# **DATA FILTERING**
## Agenda: 
Data reduction down to ~100K rows, for a decent dataframe volume, shape, & a faster run in Google Data Studio.

## Guidelines:
- **Minimum Android** is the minimum android version which allowes operation for the app. The mose popular values were found: 4.1\4.0.3\6.0 and up & Varies with device.
- **Size** must exceed 1MB.
- **Currency** will be USD only.
- **Last updated** during 2020.


In [155]:
#Coloumn labels space fix
df.columns = [x.replace(" ", "_") for x in df.columns]

#Rows SELECT
df = df.query('(Minimum_Android == "4.1 and up" | Minimum_Android == "4.0.3 and up" | Minimum_Android == "Varies with device" | \
Minimum_Android == "6.0 and up") & not Size.str.endswith("k") & Currency == "USD" & Last_Updated.str.contains("2020")',engine='python')

#Coloumns FILTER
df = df.filter(items=['App_Name', 'Category', 'Rating', 'Rating_Count', 'Installs', 'Minimum_Installs','Maximum_Installs', 'Price', 'Size','Released','Last_Updated','Ad_Supported', 'In_App_Purchases'])

#Early null creation
df['Size'] = df['Size'].replace(to_replace="Varies with device", #turning "Varies with device" to null.
                                   value = np.nan)
#Saving a copy of dataframe
dfcopy = df.copy(deep=True)


In [156]:
print("Dataframe reduced shape is", np.shape(df))
df.sample(10)

Dataframe reduced shape is (112092, 13)


,App_Name,Category,Rating,Rating_Count,Installs,Minimum_Installs,Maximum_Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases
169078,Drum Set Rhythms,Music & Audio,4.6,29.0,"5,000+",5000.0,5924,0.00,NaN,"Feb 1, 2020",06/02/2020 15:00,True,False
413089,Pakistan Radio Stations Online - Pakistan FM AM,Music & Audio,4.4,112.0,"10,000+",10000.0,38095,0.00,5.4M,"Mar 18, 2018",12/03/2020 13:22,True,False
226955,Garden Dual Photo Frames,Photography,4.0,2933.0,"1,000,000+",1000000.0,3211418,0.00,3.0M,"May 23, 2017",05/07/2020 17:32,True,False
563287,UGC NET PAPER 2 COMMERCE SOLVED PREVIOUS PAPERS,Education,0.0,0.0,"1,000+",1000.0,1130,0.00,9.0M,"Jan 20, 2020",21/01/2020 7:15,True,False
137210,Cube Icon Pack 💢Paid💢,Personalization,4.7,10.0,100+,100.0,168,1.99,2.3M,"Mar 6, 2018",18/05/2020 13:07,False,False
372212,MySFL,Sports,3.9,97.0,"10,000+",10000.0,12152,0.00,40M,"Jan 11, 2013",13/02/2020 19:56,False,False
312403,Lagu Rohani Batak OFFLINE,Music & Audio,0.0,0.0,500+,500.0,512,0.00,37M,"May 25, 2020",25/05/2020 13:59,True,False
163457,Математика - школьный курс. Подготовка ЕГЭ и ОГЭ.,Education,4.5,17115.0,"500,000+",500000.0,925973,0.00,2.9M,"Apr 18, 2015",31/01/2020 21:05,False,False
328538,Loop - Caribbean Local News,News & Magazines,4.2,5312.0,"1,000,000+",1000000.0,1713409,0.00,4.1M,"Apr 15, 2014",21/07/2020 6:53,True,False
454043,Radios Cristianas del Salvador emisoras cristi...,Music & Audio,4.5,69.0,"10,000+",10000.0,13182,0.00,6.1M,"Apr 24, 2017",14/02/2020 7:08,True,False


# **CHECKPOINT DATA INFO**

In [157]:
print("Dataframe NULL count:")
print(df.isnull().sum(axis = 0), end="\n\n")

df.info()

Dataframe NULL count:
App_Name                1
Category                0
Rating               1880
Rating_Count         1880
Installs                0
Minimum_Installs        0
Maximum_Installs        0
Price                   0
Size                10823
Released             6338
Last_Updated            0
Ad_Supported            0
In_App_Purchases        0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112092 entries, 20 to 603046
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   App_Name          112091 non-null  object 
 1   Category          112092 non-null  object 
 2   Rating            110212 non-null  float64
 3   Rating_Count      110212 non-null  float64
 4   Installs          112092 non-null  object 
 5   Minimum_Installs  112092 non-null  float64
 6   Maximum_Installs  112092 non-null  int64  
 7   Price             112092 non-null  float64
 8   Size              101269 non-

# **DATA CLEANING & OPTIMIZATION**
- A null value that was found in **App_Name** is not acceptable for further analysis, also 4 apps named: "#NAME?" should be cleaned.
- The null values in **Rating, Rating_Count, Size, and Released** columns will be replaced with mean values.
- Column dtypes will be corrected if necessary.
- Column values type will be corrected if necessary.
- Adding "Ad_bool" column, which is identical to Ad_Suppored but with 0 and 1 values, for aggregation actions such as Sum. 

In [158]:
#NULL Extraction
df = df.query('not App_Name.isnull() & not App_Name=="#NAME?"',engine='python')

#VITAL column fixes before null replacements:
#Trimming 'M' from all values in order to set as int.
df['Size'] = df['Size'].str.replace('M',"") 
df['Size'] = df['Size'].astype({'Size': 'float32'})
#Transform of "Released" coloumn to Datetime for the use of pandas.DatetimeIndex.mean
df['Released'] = pd.to_datetime(df['Released']) 

# Function replaces np.nan values in all of the data frame with relevant statistics \ dal7collab
def rep_missings(data_frame):
    d = data_frame.copy(deep = True)
    for c in data_frame:
        if data_frame[c].isnull().values.any() :
          m = data_frame[c].dropna().mean()
          d[c].fillna(m, inplace = True)
    return d;

# Activating the function
df = rep_missings(df)

In [159]:
#Adding Ad_bool column, as described above.
df['Ad_Supported'] = df['Ad_Supported'].astype({'Ad_Supported': 'string'})
df['Ad_bool']= df['Ad_Supported']
df['Ad_bool']= df['Ad_bool'].str.replace("True", "1")
df['Ad_bool']= df['Ad_bool'].str.replace("False", "0")

#Coloumn types correction
df['App_Name'] = df['App_Name'].astype({'App_Name': 'string'})
df['Category'] = df['Category'].astype({'Category': 'string'})
df['Rating'] = df['Rating'].astype({'Rating': 'float32'})
df['Rating_Count'] = df['Rating_Count'].astype({'Rating_Count': 'int64'})
df['Installs'] = df['Installs'].astype({'Installs': 'string'})
df['Minimum_Installs'] = df['Minimum_Installs'].astype({'Minimum_Installs': 'int64'})
df['Price'] = df['Price'].astype({'Price': 'float32'})
df['Released'] = pd.to_datetime(df['Released']).dt.strftime('%Y-%m-%d') #Fetching only yyyy-mm-dd from the full datetime.
df['Last_Updated'] = pd.to_datetime(df['Last_Updated']).dt.strftime('%Y-%m-%d') #Fetching only yyyy-mm-dd from the full datetime.
df['Size'] = df['Size'].astype({'Size': 'float32'})
df['Ad_bool'] = df['Ad_bool'].astype({'Ad_bool': 'int8'})


print("Dataframe final shape is", np.shape(df))
df.sample(10)

Dataframe final shape is (112087, 14)


,App_Name,Category,Rating,Rating_Count,Installs,Minimum_Installs,Maximum_Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases,Ad_bool
532180,"Telangana Board Result 2020, TS Board Inter & SSC",Education,3.8,348,"100,000+",100000,136037,0.0,8.100000,2018-02-15,2020-12-06,True,False,1
247943,Happy Janamashtami Wishes 2020,Lifestyle,0.0,0,10+,10,20,0.0,3.400000,2015-08-30,2020-07-13,True,False,1
353704,Miner World : Grow Miner,Adventure,3.9,1062,"100,000+",100000,252848,0.0,4.600000,2018-11-08,2020-06-27,True,False,1
394030,اسماء بنات و أولاد (أنمي و مانغا) 2020 - بدون نت,Lifestyle,4.3,687,"50,000+",50000,62595,0.0,49.000000,2017-07-23,2020-03-24,True,False,1
507078,Speed Rush Las Vegas Free Slot,Casino,4.4,190,"10,000+",10000,21550,0.0,7.400000,2015-06-12,2020-01-14,True,True,1
487993,Share Up - File Transfer & Sharing,Communication,5.0,6,100+,100,219,0.0,65.000000,2020-07-11,2020-07-14,True,False,1
253558,Hi MATE 원격제어(현대건설기계),Entertainment,0.0,0,100+,100,293,0.0,3.500000,2017-10-13,2020-07-15,False,False,0
250853,Health Tips Telugu (Arogya Chitkalu),Health & Fitness,4.1,10,"10,000+",10000,13024,0.0,3.500000,2018-08-13,2020-04-24,True,False,1
353508,Mindbody: Home Workout & Fitness App,Health & Fitness,4.6,13230,"1,000,000+",1000000,3918915,0.0,18.566137,2013-10-04,2020-05-21,False,False,0
105695,Card Game 29,Card,4.2,33619,"5,000,000+",5000000,8882980,0.0,17.000000,2014-09-02,2020-07-20,True,False,1


# **POSTPROCESSING DATA INFO**

In [160]:
#Genereal information after preprocessing
print(df.info(),end = '\n\n')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112087 entries, 20 to 602972
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   App_Name          112087 non-null  string 
 1   Category          112087 non-null  string 
 2   Rating            112087 non-null  float32
 3   Rating_Count      112087 non-null  int64  
 4   Installs          112087 non-null  string 
 5   Minimum_Installs  112087 non-null  int64  
 6   Maximum_Installs  112087 non-null  int64  
 7   Price             112087 non-null  float32
 8   Size              112087 non-null  float32
 9   Released          112087 non-null  object 
 10  Last_Updated      112087 non-null  object 
 11  Ad_Supported      112087 non-null  string 
 12  In_App_Purchases  112087 non-null  bool   
 13  Ad_bool           112087 non-null  int8   
dtypes: bool(1), float32(3), int64(3), int8(1), object(2), string(4)
memory usage: 10.0+ MB
None



# **EXPORT TO GOOGLE DRIVE**


In [161]:
#Exporting a csv into google drive
#df.to_csv('/content/drive/My Drive/GooglePlay-Store_Optimized_v15.csv')

# **STATISTICS**

In [162]:
#Retrieve coloumn general info 
name = 'App_Name'
print(df[name].describe())
print("Coloumn's real dtype is:", df[name].dtypes)

count                                                112087
unique                                               112087
top       ТЬМА ПОГЛОТИЛА ВАС - мрачный пиксельный платфо...
freq                                                      1
Name: App_Name, dtype: object
Coloumn's real dtype is: string


In [163]:
#Corrections for BI Robot operations
df['App_Name'] = df['App_Name'].astype({'App_Name': 'object'})
df['Category'] = df['Category'].astype({'Category': 'object'})

In [166]:
# BI Insights Robot
def bi_robot(dataframe, dimensions, measure):
    t = {}
    for d in dimensions:
        if (np.issubdtype(dataframe[d].dtype, np.number)):
            t[d] = df[d].describe()[7]    #Max value of a numeric variable
        
        else :
            t[d] = df[d].describe()[2]    #The most frequent value of any other variable

    for v in t:
        key       = v
        value     = [t[v]]
        dataframe = dataframe.loc[dataframe[key].isin(value)] 
        stat      = dataframe[measure].mean()

    print('---------------------')
    print(t, end = '')  
    print(' Mean '+ measure +': ' + "{:.2f}".format(stat))
    print('---------------------') 

    return t

In [167]:
df_ins         = df.copy(deep = True)
filters    = ['Category']
measure    = 'Rating'
df_ins = bi_robot(df, filters, measure)
df_ins

---------------------
{'Category': 'Education'} Mean Rating: 3.16
---------------------


{'Category': 'Education'}